### Year 0 Topology 0 Contingency Analysis

In [1]:
import pandas as pd
import numpy as np

AC contingency calculation was conducted on the hypothetical SAVNW system with saved case file savnw.sav
Configuration files used are 
1.	Subsystem file savnw.sub<br>
The subsystem file gives the data about the subsystems studied <br>

![savnw subsystem file](sub.png)
2.	Monitor file savnw.mon<br>
The monitored data file gives the monitored parameters<br>

![savnw monitor file](mon.png)
3.	Contingency file savnw.con<br>
The contingency file gives the tested contingencies <br>

![savnw contingency file](con.png)

The API DFAX_2 is used to construct the Distribution Factor Data File corresponding to the .sav, .sub, .mon, .con files. The constructed distribution factor data file is savnw.dfx<br>. By running the AC contingency calculation function ACCC_WITH_DSP_3, the Contingency Solution Output .acc file obtained is savnw.acc. <br>

Python code to conduct AC contingency calculation is

Using the contingency solution output file savnw.acc, the results is exported as an excel file for further analysis. The results exported are ACCC Analysis Summary, Monitored Branch Flow (MVA), Monitored Bus Voltage. <br>

Python code to export AC contingency solution output file as excel is 

In the following sections the exported data is imported as a python Dataframe to extract non-converged contingencies, branch overloads and bus upper and lower voltage range violations.  

#### Non-converged contingencies 
Summary sheet in the ACCC file is used to extract all the contingencies tested. The converged contingencies are extracted either from the Bus Voltage sheet or from the Branch Flow sheet. Difference between the two lists gives the desired non-converged contingency. 
Here the branch flow sheet is used to extract the converged contingencies 

In [2]:
# out.xlsx file has export flows for all elements
data_f = pd.read_excel('out.xlsx', sheet_name='Branch Flow')
data_flow = data_f.dropna(how='all').reset_index(drop=True)
data_sum = pd.read_excel('out.xlsx', sheet_name='Summary')
data_summary= data_sum.dropna(how='all').reset_index(drop=True)
data_cont = data_summary.loc[132:138].reset_index(drop=True)
# extracting all the tested contingencies 
tested_conts = list(data_cont['ACCC SOLUTION SUMMARY'])
ans_list = "".join(tested_conts)
cont_tested = ans_list.split(',')
cont_test = [s.strip() for s in cont_tested]
scen_cont = data_flow['CONTINGENCY'].unique()

# extracting contingencies for the studied scenario
scen_cont = data_flow['CONTINGENCY'].unique()
# blown up scenarios 
print(f'{len(scen_cont)} contingencies were converged out of the {len(cont_test)} contingencies tested, the contingencies not converged are', ', '.join(list(set(cont_test)-set(scen_cont))))


42 contingencies were converged out of the 47 contingencies tested, the contingencies not converged are UNIT 206(1), SING OPN LIN   14 205-206(1), BUS 152, BUS 154, BUS 201


#### Branch flow overload

In [3]:
data_foverload = pd.read_excel('savnwacc.xlsx', sheet_name='Branch Flow')
data_flow_overload = data_foverload.dropna(how='all').reset_index(drop=True)
data_flow_overload.drop(columns = 'Unnamed: 6').dropna(how='all').reset_index(drop=True)

,BRANCH,CONTINGENCY,MVAFLOW,AMPFLOW,RATE RATE1/RATE2,% FLOW
0,153 MID230 230.00 154 DOWNT...,SING OPN LIN 11 202-203(1),-349.966949,359.164276,350.0,102.618365
1,3001 MINE 230.00 3003 S. MI...,BUS 151,616.276855,599.615662,300.0,199.871887
2,154 DOWNTN 230.00 203 EAST2...,BUS 205,266.194733,294.191284,250.0,117.676514


Branches reporting overload are 

In [4]:
list([s.strip() for s in data_foverload['BRANCH'].dropna().unique()])

['153     MID230      230.00    154     DOWNTN      230.00 1',
 '3001     MINE        230.00   3003     S. MINE     230.00 1',
 '154     DOWNTN      230.00    203     EAST230     230.00 1']

#### Bus voltage range violations

In [5]:
data_v = pd.read_excel('savnwacc.xlsx', sheet_name='Bus Voltage', usecols = ['BUS', 'RECORD', 'TYPE', 'MIN/DROP', 'MAX/RISE', 'CONTINGENCY',
       'BASE VOLTS', 'CONT VOLTS', 'DEVIATION', 'RANGE VIO', 'DEV VIO'] )
data_volt = data_v.dropna(how='all').reset_index(drop=True)

##### Bus voltage upper range violation (Bus Voltage > 1.1 PU)

In [6]:
data_high = data_volt[data_volt['CONT VOLTS']>1.1].reset_index(drop=True)
data_high

,BUS,RECORD,TYPE,MIN/DROP,MAX/RISE,CONTINGENCY,BASE VOLTS,CONT VOLTS,DEVIATION,RANGE VIO,DEV VIO
0,201 HYDRO 500.00,ALL,RANGE,0.95,1.05,BUS 151,1.057659,1.1014,0.043741,0.0514,NaN
1,201 HYDRO 500.00,LIMIT,RANGE,0.90,1.10,BUS 151,1.057659,1.1014,0.043741,0.0014,NaN


Bus reporting voltage upper range violation is 

In [7]:
', '.join([s.strip().split(' ')[0] for s in list(data_high['BUS'].unique())])

'201'

##### Bus voltage lower range violation (Bus Voltage < 0.9 PU)

In [8]:
data_low = data_volt[data_volt['CONT VOLTS']<0.9].reset_index(drop=True)
data_low

,BUS,RECORD,TYPE,MIN/DROP,MAX/RISE,CONTINGENCY,BASE VOLTS,CONT VOLTS,DEVIATION,RANGE VIO,DEV VIO


There are no buses reporting voltage lower range violations. 

#### Conclusion

Year 0 Topology 0 contingency violation analysis was carried out to extract the following 
1. Branches loaded more than 100% of their rating 
2. Buses violating upper range voltage of 1.1 PU
3. Buses violating lower range voltage of 0.9 PU